In [1]:
import pandas as pd
import numpy as np

In [2]:
player_attributes = pd.read_csv('Player_Attributes.csv')
matches_epl = pd.read_csv('Matches_EPL.csv')

In [3]:

players_df = pd.read_csv('data/player_attributes_transformed.csv')
                         #dtype={"player_api_id":int,"year":int,"player_fifa_api_id":int,"overall_rating":float})

In [4]:
#Function to populate result column for each match row
def get_results(home_goal,away_goal):
    
    #Win or Defeat on the basis of home team
    if home_goal > away_goal:
        return "Win"
    elif home_goal == away_goal:
        return "Draw"
    else:
        return "Defeat"
        

In [5]:
#Creating a column result from the final scoreline
matches_epl['result'] = matches_epl.apply(lambda x: get_results(x['home_team_goal'],x['away_team_goal']),axis=1)

In [6]:
home_players_list = ['home_player_1','home_player_2','home_player_3','home_player_4','home_player_5','home_player_6','home_player_7','home_player_8','home_player_9','home_player_10','home_player_11']
away_players_list = ['away_player_1','away_player_2','away_player_3','away_player_4','away_player_5','away_player_6','away_player_7','away_player_8','away_player_9','away_player_10','away_player_11']

In [7]:
#Changing Dtype of ids in the dataframe and handling NaNs
matches_epl[home_players_list] = matches_epl[home_players_list].fillna(-1)
matches_epl[home_players_list] = matches_epl[home_players_list].astype(int)

matches_epl[away_players_list] = matches_epl[away_players_list].fillna(-1)
matches_epl[away_players_list] = matches_epl[away_players_list].astype(int)

In [8]:
#Changing date format in dataframe from string to datetime
matches_epl['date'] = pd.to_datetime(matches_epl['date'], format="%Y-%m-%d")

In [9]:
# for player in home_players_list:
#     print(player, "no. of nulls", matches_epl[player].isnull().sum())
# print('---------------')
# for player in away_players_list:
    
#     print(player, "no. of nulls", matches_epl[player].isnull().sum())


In [10]:
#Getting team squad api ids from multiple columns into one list
matches_epl['home_players_ids']= matches_epl[home_players_list].values.tolist()
matches_epl['away_players_ids']= matches_epl[away_players_list].values.tolist()

In [11]:
matches_epl['season'] = matches_epl['season'].apply(lambda x: int(x.split('/')[0]))

In [12]:
def get_team_ratings(ids_list,season):
    team_rating = []
    for player_id in ids_list:
        if(player_id!=-1):
            #Ignoring null values which were converted to -1 in the previous code
            rating = players_df[(players_df['player_api_id']==player_id) & (players_df['year']==season)]['overall_rating']
            if not rating.empty:
                #If player data for that season exists, fetch the rating of player for that season
                team_rating.append(rating.iloc[0])
    team_rating = sum(team_rating)/len(team_rating)
    return team_rating


In [13]:
matches_epl['home_rating'] = matches_epl.apply(lambda x: get_team_ratings(x['home_players_ids'],x['season']), axis=1)
matches_epl['away_rating'] = matches_epl.apply(lambda x: get_team_ratings(x['away_players_ids'],x['season']), axis=1)

In [14]:
#retrieving only important columns for the model
matches_columns = ['id','season','date','match_api_id','home_team_api_id','away_team_api_id',\
                   'home_team_goal','away_team_goal',\
                   'home_rating','away_rating','result']

In [15]:
matches_epl = matches_epl[matches_columns]
temp_df = matches_epl[matches_epl['season']==2008]

In [16]:
home_teams = temp_df.home_team_api_id.unique()
home_teams

array([10260,  9825,  8472,  8654, 10252,  8668,  8549,  8559,  8667,
        8455, 10194,  8650, 10261,  9879,  8586,  8659,  8462,  8456,
        8655,  8528], dtype=int64)

In [17]:
away_teams = temp_df.away_team_api_id.unique()
away_teams

array([10261,  8659,  8650,  8528,  8456,  8655,  8586, 10194,  9879,
        8462,  8654,  8668,  8455,  8472,  8667, 10252,  9825, 10260,
        8549,  8559], dtype=int64)

In [18]:
matches_epl = matches_epl.sort_values(by='date',ascending=False).reset_index(drop=False)

In [22]:
matches_epl[(matches_epl['home_team_api_id']==8455)]

,index,id,season,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_rating,away_rating,result
8,2971,4700,2015,2016-05-15,1987598,8455,8197,1,1,81.815702,71.847619,Draw
24,2951,4680,2015,2016-05-02,1989061,8455,8586,2,2,82.443506,78.987234,Draw
54,2932,4661,2015,2016-04-16,1989042,8455,8456,0,3,79.760724,82.540404,Defeat
82,2900,4629,2015,2016-03-19,1989010,8455,8654,2,2,81.172727,75.666667,Draw
99,2870,4599,2015,2016-03-05,1988990,8455,10194,1,1,81.164187,76.057576,Draw
127,2843,4572,2015,2016-02-13,1988963,8455,10261,5,1,83.820779,75.601010,Win
131,2832,4561,2015,2016-02-07,1988952,8455,10260,1,1,82.381385,77.005119,Draw
168,2802,4531,2015,2016-01-16,1988922,8455,8668,3,3,82.290476,77.995671,Draw
172,2794,4523,2015,2016-01-13,1988914,8455,8659,2,2,82.308658,74.501732,Draw
203,2752,4481,2015,2015-12-26,1988882,8455,9817,2,2,83.420779,73.941991,Draw


In [23]:
import numpy
def get_last_five_score(team,date):
    streak_score=0
    last_five = matches_epl[((matches_epl['home_team_api_id']==team) | (matches_epl['away_team_api_id']==team)) & (matches_epl['date']<date)].head(5)
#     print(len(last_five))
#     last_five['team_ground'] = last_five.apply(lambda x: 'home' if x['home_team_api_id']==team else 'away', axis=1)
    #Last 5 not empty - i.e. no previous match data for this team id
    if not last_five.empty:
#         #To determine whether the last match of team was at home or away to get the score difference 
#         last_five['team_score'] = 
        last_five['team_score'] = last_five.apply(lambda x: (x['home_team_goal']-x['away_team_goal'] if (x['home_team_api_id']==team) else x['away_team_goal']-x['home_team_goal'] ),axis=1)
        streak_score = last_five.team_score.sum()
#         print('=--------',date)
#         print(last_five)
#         print(streak_score)
    
        if type(streak_score)!= numpy.int64:
            return streak_score.values[0]
        else:
            return streak_score
    else:
        
        print(date,'--------------------------------Empty')
        return 0

In [24]:
matches_epl['home_last_five_score'] = matches_epl.apply(lambda x: get_last_five_score(x['home_team_api_id'],x['date']),axis=1)

2015-08-08 00:00:00 --------------------------------Empty
2014-08-16 00:00:00 --------------------------------Empty
2013-08-18 00:00:00 --------------------------------Empty
2012-08-18 00:00:00 --------------------------------Empty
2011-08-13 00:00:00 --------------------------------Empty
2009-08-15 00:00:00 --------------------------------Empty
2008-08-17 00:00:00 --------------------------------Empty
2008-08-17 00:00:00 --------------------------------Empty
2008-08-17 00:00:00 --------------------------------Empty
2008-08-16 00:00:00 --------------------------------Empty
2008-08-16 00:00:00 --------------------------------Empty
2008-08-16 00:00:00 --------------------------------Empty
2008-08-16 00:00:00 --------------------------------Empty
2008-08-16 00:00:00 --------------------------------Empty
2008-08-16 00:00:00 --------------------------------Empty
2008-08-16 00:00:00 --------------------------------Empty


In [26]:
matches_epl['away_last_five_score'] = matches_epl.apply(lambda x: get_last_five_score(x['away_team_api_id'],x['date']),axis=1)

2015-08-08 00:00:00 --------------------------------Empty
2013-08-17 00:00:00 --------------------------------Empty
2012-08-19 00:00:00 --------------------------------Empty
2011-08-15 00:00:00 --------------------------------Empty
2011-08-13 00:00:00 --------------------------------Empty
2010-08-14 00:00:00 --------------------------------Empty
2009-08-16 00:00:00 --------------------------------Empty
2009-08-15 00:00:00 --------------------------------Empty
2008-08-17 00:00:00 --------------------------------Empty
2008-08-17 00:00:00 --------------------------------Empty
2008-08-17 00:00:00 --------------------------------Empty
2008-08-16 00:00:00 --------------------------------Empty
2008-08-16 00:00:00 --------------------------------Empty
2008-08-16 00:00:00 --------------------------------Empty
2008-08-16 00:00:00 --------------------------------Empty
2008-08-16 00:00:00 --------------------------------Empty
2008-08-16 00:00:00 --------------------------------Empty
2008-08-16 00:

In [25]:
matches_epl[(matches_epl['home_team_api_id']==8455) |(matches_epl['away_team_api_id']==8455)]

,index,id,season,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_rating,away_rating,result,home_last_five_score
8,2971,4700,2015,2016-05-15,1987598,8455,8197,1,1,81.815702,71.847619,Draw,-1
12,2894,4623,2015,2016-05-11,1989004,8650,8455,1,1,79.077020,81.171763,Draw,5
23,2967,4696,2015,2016-05-07,1989077,8472,8455,3,2,75.132576,83.320779,Win,1
24,2951,4680,2015,2016-05-02,1989061,8455,8586,2,2,82.443506,78.987234,Draw,3
40,2941,4670,2015,2016-04-23,1989051,8678,8455,1,4,69.701515,82.074360,Defeat,-6
54,2932,4661,2015,2016-04-16,1989042,8455,8456,0,3,79.760724,82.540404,Defeat,4
63,2926,4655,2015,2016-04-09,1989036,10003,8455,1,0,77.053030,77.503960,Win,2
70,2911,4640,2015,2016-04-02,1989021,10252,8455,0,4,74.962554,76.617596,Defeat,-10
82,2900,4629,2015,2016-03-19,1989010,8455,8654,2,2,81.172727,75.666667,Draw,6
99,2870,4599,2015,2016-03-05,1988990,8455,10194,1,1,81.164187,76.057576,Draw,6


In [28]:
def get_last_five_opponent(team,date,opponent):
    last_five = matches_epl[((matches_epl['home_team_api_id']==team) & (matches_epl['away_team_api_id']==opponent)) & (matches_epl['date']<date)].head(5)
    if not last_five.empty:
        streak_score = last_five.groupby('id').apply(lambda x: x['home_team_goal']-x['away_team_goal']).sum()
        if type(streak_score)!= numpy.int64:
            return streak_score.values[0]
        else:
            return streak_score
    else:
        
        print(date,'--------------------------------Empty')
        return 0

In [29]:
#Stores last 5 games' performance of a team vs an opponent at home - to factor in home advantage
matches_epl['home_opponent_score'] = matches_epl.apply(lambda x: get_last_five_opponent(x['home_team_api_id'],x['date'], x['away_team_api_id']),axis=1)

2016-05-17 00:00:00 --------------------------------Empty
2016-05-15 00:00:00 --------------------------------Empty
2016-05-11 00:00:00 --------------------------------Empty
2016-05-08 00:00:00 --------------------------------Empty
2016-05-07 00:00:00 --------------------------------Empty
2016-04-30 00:00:00 --------------------------------Empty
2016-04-30 00:00:00 --------------------------------Empty
2016-04-23 00:00:00 --------------------------------Empty
2016-04-20 00:00:00 --------------------------------Empty
2016-04-17 00:00:00 --------------------------------Empty
2016-04-16 00:00:00 --------------------------------Empty
2016-04-09 00:00:00 --------------------------------Empty
2016-04-09 00:00:00 --------------------------------Empty
2016-04-02 00:00:00 --------------------------------Empty
2016-04-02 00:00:00 --------------------------------Empty
2016-03-20 00:00:00 --------------------------------Empty
2016-03-19 00:00:00 --------------------------------Empty
2016-03-12 00:

2014-03-22 00:00:00 --------------------------------Empty
2014-03-22 00:00:00 --------------------------------Empty
2014-03-15 00:00:00 --------------------------------Empty
2014-03-15 00:00:00 --------------------------------Empty
2014-03-08 00:00:00 --------------------------------Empty
2014-03-08 00:00:00 --------------------------------Empty
2014-03-02 00:00:00 --------------------------------Empty
2014-03-02 00:00:00 --------------------------------Empty
2014-02-22 00:00:00 --------------------------------Empty
2014-02-22 00:00:00 --------------------------------Empty
2014-02-11 00:00:00 --------------------------------Empty
2014-02-11 00:00:00 --------------------------------Empty
2014-02-08 00:00:00 --------------------------------Empty
2014-02-08 00:00:00 --------------------------------Empty
2014-02-02 00:00:00 --------------------------------Empty
2014-02-01 00:00:00 --------------------------------Empty
2014-01-28 00:00:00 --------------------------------Empty
2014-01-28 00:

2012-08-19 00:00:00 --------------------------------Empty
2012-08-18 00:00:00 --------------------------------Empty
2012-05-13 00:00:00 --------------------------------Empty
2012-05-13 00:00:00 --------------------------------Empty
2012-05-13 00:00:00 --------------------------------Empty
2012-05-06 00:00:00 --------------------------------Empty
2012-05-06 00:00:00 --------------------------------Empty
2012-05-05 00:00:00 --------------------------------Empty
2012-04-29 00:00:00 --------------------------------Empty
2012-04-28 00:00:00 --------------------------------Empty
2012-04-28 00:00:00 --------------------------------Empty
2012-04-21 00:00:00 --------------------------------Empty
2012-04-21 00:00:00 --------------------------------Empty
2012-04-21 00:00:00 --------------------------------Empty
2012-04-14 00:00:00 --------------------------------Empty
2012-04-14 00:00:00 --------------------------------Empty
2012-04-14 00:00:00 --------------------------------Empty
2012-04-11 00:

2010-11-06 00:00:00 --------------------------------Empty
2010-11-01 00:00:00 --------------------------------Empty
2010-10-23 00:00:00 --------------------------------Empty
2010-10-17 00:00:00 --------------------------------Empty
2010-10-03 00:00:00 --------------------------------Empty
2010-09-25 00:00:00 --------------------------------Empty
2010-09-19 00:00:00 --------------------------------Empty
2010-09-18 00:00:00 --------------------------------Empty
2010-09-11 00:00:00 --------------------------------Empty
2010-08-28 00:00:00 --------------------------------Empty
2010-08-28 00:00:00 --------------------------------Empty
2010-08-21 00:00:00 --------------------------------Empty
2010-08-14 00:00:00 --------------------------------Empty
2010-05-09 00:00:00 --------------------------------Empty
2010-05-09 00:00:00 --------------------------------Empty
2010-05-09 00:00:00 --------------------------------Empty
2010-05-01 00:00:00 --------------------------------Empty
2010-05-01 00:

2009-05-02 00:00:00 --------------------------------Empty
2009-04-27 00:00:00 --------------------------------Empty
2009-04-26 00:00:00 --------------------------------Empty
2009-04-26 00:00:00 --------------------------------Empty
2009-04-25 00:00:00 --------------------------------Empty
2009-04-25 00:00:00 --------------------------------Empty
2009-04-25 00:00:00 --------------------------------Empty
2009-04-25 00:00:00 --------------------------------Empty
2009-04-25 00:00:00 --------------------------------Empty
2009-04-25 00:00:00 --------------------------------Empty
2009-04-25 00:00:00 --------------------------------Empty
2009-04-22 00:00:00 --------------------------------Empty
2009-04-22 00:00:00 --------------------------------Empty
2009-04-21 00:00:00 --------------------------------Empty
2009-04-19 00:00:00 --------------------------------Empty
2009-04-19 00:00:00 --------------------------------Empty
2009-04-18 00:00:00 --------------------------------Empty
2009-04-18 00:

2008-12-28 00:00:00 --------------------------------Empty
2008-12-28 00:00:00 --------------------------------Empty
2008-12-26 00:00:00 --------------------------------Empty
2008-12-26 00:00:00 --------------------------------Empty
2008-12-26 00:00:00 --------------------------------Empty
2008-12-26 00:00:00 --------------------------------Empty
2008-12-26 00:00:00 --------------------------------Empty
2008-12-26 00:00:00 --------------------------------Empty
2008-12-26 00:00:00 --------------------------------Empty
2008-12-26 00:00:00 --------------------------------Empty
2008-12-26 00:00:00 --------------------------------Empty
2008-12-26 00:00:00 --------------------------------Empty
2008-12-22 00:00:00 --------------------------------Empty
2008-12-21 00:00:00 --------------------------------Empty
2008-12-21 00:00:00 --------------------------------Empty
2008-12-21 00:00:00 --------------------------------Empty
2008-12-20 00:00:00 --------------------------------Empty
2008-12-20 00:

2008-09-21 00:00:00 --------------------------------Empty
2008-09-21 00:00:00 --------------------------------Empty
2008-09-20 00:00:00 --------------------------------Empty
2008-09-20 00:00:00 --------------------------------Empty
2008-09-20 00:00:00 --------------------------------Empty
2008-09-20 00:00:00 --------------------------------Empty
2008-09-20 00:00:00 --------------------------------Empty
2008-09-15 00:00:00 --------------------------------Empty
2008-09-14 00:00:00 --------------------------------Empty
2008-09-13 00:00:00 --------------------------------Empty
2008-09-13 00:00:00 --------------------------------Empty
2008-09-13 00:00:00 --------------------------------Empty
2008-09-13 00:00:00 --------------------------------Empty
2008-09-13 00:00:00 --------------------------------Empty
2008-09-13 00:00:00 --------------------------------Empty
2008-09-13 00:00:00 --------------------------------Empty
2008-09-13 00:00:00 --------------------------------Empty
2008-08-31 00:

In [32]:
matches_epl[(matches_epl['home_team_api_id']==8455) & (matches_epl['away_team_api_id']==10260)]

,index,id,season,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_rating,away_rating,result,home_last_five_score,away_last_five_score,home_opponent_score
131,2832,4561,2015,2016-02-07,1988952,8455,10260,1,1,82.381385,77.005119,Draw,4,4,3
439,2542,4271,2014,2015-04-18,1724304,8455,10260,1,0,82.011039,79.283838,Win,4,9,3
922,2041,3770,2013,2014-01-19,1474923,8455,10260,3,1,81.766811,78.591017,Win,7,5,1
1432,1892,3621,2012,2012-10-28,1229187,8455,10260,2,3,82.227273,80.727273,Defeat,7,9,2
1662,1301,3030,2011,2012-02-05,1025473,8455,10260,3,3,82.045455,82.050000,Draw,0,2,2
2003,859,2588,2010,2011-03-01,839993,8455,10260,2,1,83.727273,82.136364,Win,7,7,1
2545,419,2148,2009,2009-11-08,658728,8455,10260,1,0,84.227273,81.454545,Win,8,3,0
2992,339,2068,2008,2008-09-21,489091,8455,10260,1,1,84.500000,81.772727,Draw,7,0,0


In [37]:
matches_epl[(matches_epl['home_team_api_id']==10194) & (matches_epl['away_team_api_id']==8586)]

,index,id,season,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_rating,away_rating,result,home_last_five_score,away_last_five_score,home_opponent_score
46,2938,4667,2015,2016-04-18,1989048,10194,8586,0,4,76.765152,77.916780,Defeat,-3,8,1
407,2579,4308,2014,2015-05-09,1724341,10194,8586,3,0,74.036580,76.157969,Win,-2,0,-3
789,2196,3925,2013,2014-04-26,1475093,10194,8586,0,1,75.015476,77.694697,Defeat,2,3,-1
1152,1828,3557,2012,2013-05-12,1229501,10194,8586,1,2,75.500000,80.409091,Defeat,-1,3,0
1752,1206,2935,2011,2011-12-11,1024952,10194,8586,2,1,74.954545,81.136364,Win,-5,11,-1
2264,878,2607,2010,2010-08-21,839832,10194,8586,1,2,73.954545,79.409091,Defeat,-11,-2,0
2361,629,2358,2009,2010-03-20,659128,10194,8586,1,2,73.227273,76.590909,Defeat,-1,5,1
2962,369,2098,2008,2008-10-19,489121,10194,8586,2,1,69.090909,79.590909,Win,-5,-4,0


In [39]:
matches_epl['final_rating'] = matches_epl['home_rating'] - matches_epl['away_rating']

In [74]:
matches_epl.to_csv('Matches_EPL_processed.csv')